In [ ]:
import os
import numpy as np
import torch
from torch import Tensor
from torch.nn.parameter import Parameter
from torch.autograd import Variable
import torch.nn.functional as F

from lib.parts_dataset import PartsDataset,DatasetSplit
from lib.parts_model import PartsModel
from lib.opt import optimize

import lib.media as media
from IPython.display import Audio
from scipy.io import wavfile

In [ ]:
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'   # see issue #152
os.environ['CUDA_VISIBLE_DEVICES']='0'
#os.environ['CUDA_LAUNCH_BLOCKING']='1'

checkpoint_dir = '_singlepart14'
context = 5

In [ ]:
train_set = PartsDataset(context=context,split=DatasetSplit.train)
test_set = PartsDataset(context=context,split=DatasetSplit.test)

In [ ]:
class Model(PartsModel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def define_graph(self, debug=False):
        self.knote = 16
        self.k = 300
        self.maxdur = train_set.maxdur
        
        # reference meter for pitch class
        self.noteref = np.zeros([1,1,2*self.m-1],dtype=np.float32)
        self.noteref[0,0,self.m] = 1
        self.noteref = Variable(torch.from_numpy(self.noteref).cuda(), requires_grad=False)
        
        fold = np.zeros([self.m,12],dtype=np.float32)
        for n in range(self.m):
            fold[n,n%12] = 1
        self.fold = Variable(torch.from_numpy(fold).cuda(),requires_grad=False)
        
        # rhythm prediction
        self.rhythm_wt = Parameter(Tensor(self.context*self.maxdur,self.k))
        self.rhythm_wn = Parameter(Tensor(12*self.context,self.k))
        self.rhythm_wloc = Parameter(Tensor(48,self.k))
        
        self.rhythm_wtop = Parameter(Tensor(self.k,self.maxdur))
        self.rhythm_bias = Parameter(Tensor(self.maxdur))
        
        # notes prediction
        self.notes_wnote = Parameter(Tensor(self.knote,1,1,2*self.m))
        
        self.notes_wt = Parameter(Tensor(self.context*self.maxdur,self.k))
        self.notes_wn = Parameter(Tensor(self.k,self.knote,self.context,1))
        self.notes_wyt = Parameter(Tensor(self.maxdur,self.k))
        self.notes_wyn = Parameter(Tensor(self.k,1,self.m-1))
        self.notes_wref = Parameter(Tensor(self.k,1,self.m))
        
        self.notes_wtop = Parameter(Tensor(self.k,1))
        self.notes_bias = Parameter(Tensor(self.m))
    
    def predict_rhythm(self, e, t, loc):
        frhythm = torch.mm(t.view(-1,self.context*self.maxdur),self.rhythm_wt)
        ef = torch.mm(e.view(-1,self.m),self.fold).view(e.shape[0],e.shape[1],-1)
        fnotes = torch.mm(ef.view(-1,self.context*12),self.rhythm_wn)
        floc = torch.mm(loc,self.rhythm_wloc)
        
        zx = F.relu(frhythm + fnotes + floc)
        return torch.mm(zx,self.rhythm_wtop) + self.rhythm_bias
    
    def predict_notes(self, e, t, y, yt):
        fembed = F.conv2d(F.pad(e,(self.m,self.m-1,0,0,0,0))[:,None],self.notes_wnote)
        
        frhythm = torch.mm(t.view(-1,self.context*self.maxdur),self.notes_wt)[:,:,None]
        fnotes = F.conv2d(fembed,self.notes_wn)[:,:,0,:]
        fcurt = torch.mm(yt,self.notes_wyt)[:,:,None]
        fcurn = F.conv1d(F.pad(y,(self.m-1,0,0,0))[:,None],self.notes_wyn)[:,:,:-1]
        fref = F.conv1d(self.noteref,self.notes_wref)
        
        zx = F.relu(frhythm + fnotes + fcurt + fcurn + fref).transpose(1,2).contiguous()
        return torch.mm(zx.view(-1,self.k),self.notes_wtop).view(-1,self.m) + self.notes_bias
    
    def forward(self, x):
        e,t,y,yt,loc = x
        return self.predict_rhythm(e,t,loc), self.predict_notes(e,t,y,yt)

In [ ]:
model = Model(checkpoint_dir, avg=.999, context_length=context,
              offset=train_set.offset, m=train_set.m, dur_map=train_set.dur_map,
              weight_scale=.01)
model.initialize()

In [ ]:
optimize(model,train_set,test_set,learning_rate=.01,batch_size=300,workers=4,update_rate=1000)

In [ ]:
optimize(model,train_set,test_set,learning_rate=.001,batch_size=300,workers=4,update_rate=1000)

In [ ]:
model.restore_checkpoint()
plt = media.PlotFormatter()
plt.plot('log loss',model.stats['ll_tr'][2],color='b')
plt.plot('log loss',model.stats['ll_test'][2],color='g',share=True)
plt.plot('avp notes',model.stats['apn_tr'][2],color='b')
plt.plot('avp notes',model.stats['apn_ts'][2],color='g',share=True)
plt.plot('rhythm weights',model.sum_weights('rhythm'),color='g')
plt.plot('notes weights',model.sum_weights('notes'),color='g')
plt.plot('rhythm log loss',model.stats['llt_tr'][2],color='b')
plt.plot('rhythm log loss',model.stats['llt_ts'][2],color='g',share=True)
plt.plot('notes log loss',model.stats['lln_tr'][2],color='b')
plt.plot('notes log loss',model.stats['lln_ts'][2],color='g',share=True)
plt.show()